In [1]:
import sys

sys.path.append('..')

from src_refactored.configs import DataConfig, NetConfig, NetArchitecture, FilterConfig
from src_refactored.utils import train, get_wandb_logger
from src_refactored.configs import PACKAGE_PATH
from src_refactored.data_processor import DataProcessor

In [2]:
filter=FilterConfig(
    n_bins= 30,
    n_gaps= 10,
    gap_size= 5, 
    rms_ratio= 0.,
    non_zero_ratio=0.8
)

data_config = DataConfig(
    path="/mnt/c/work/Fall_2021_csv",
    output_path=f"{PACKAGE_PATH}/resources/datasets",
    # class_names=["cz_3", "falcon_9", "atlas_V",  "h2a", "globalstar"],
    # regexes=[r'CZ-3B.*', r'FALCON_9.*', r'ATLAS_[5|V]_CENTAUR_R_B$',  r'H-2A.*', r'GLOBALSTAR.*'],
    class_names=["h2a", "falcon_9"],
    regexes=[r'H-2A.*', r'FALCON_9.*'],
    validation_split=0.2,
    number_of_training_examples_per_class=10_000,
    filter_config=filter,
    convert_to_mag=False,
    fourier=False,
    std= False,
    rms= False,
    residuals= False,
    amplitude= False,
    lc= True,
    reconstructed_lc= False,
    push_to_max= False
)

In [3]:
dp = DataProcessor(data_config)

In [4]:
dp.examples = None
dp.labels = None

dp.load_data_MMT()

In [5]:
dp.examples.shape, dp.labels.shape  

((4616, 334), (4616,))

In [6]:
dp.use_lc = False
dp.use_amplitude = False
dp.use_residuals = False
dp.use_fourier = False
dp.use_std = False
dp.use_rms = False
dp.use_reconstructed = True

(x_t, y_t), (x_v, y_v) = dp.prepare_dataset(dp.examples, dp.labels, 42)
x_t.shape, y_t.shape, x_v.shape, y_v.shape

[3.588769201862262, -0.058930573503567896, -0.20083687652266802, 0.05697090020176221, 0.02543013999377597, -0.028356595924175165, -0.06657053601851826, -0.045838116001001294, -0.010716163330547479, -0.11409309597885442, 0.06709762022868382, -0.015956833123647707, -0.03758323130252905, -0.06158514345065291, -0.009741592624179285, 0.014867893224390194, 0.06971103139847246]
(4616, 17) (4616, 17) (4616, 300)
(4616, 300)


((3693, 300), (3693,), (923, 300), (923,))

In [7]:
import numpy as np
x = np.random.rand(10, 4)
(np.sum(x, axis=1) / (np.sum(x <0.3,axis=1) + 1e-6)) 


array([2.26213074e+00, 1.49117198e+00, 1.44658909e+00, 6.89895253e-01,
       1.75381665e+00, 6.55907358e-01, 7.83949846e-01, 2.13310333e+00,
       2.18345930e+00, 2.85107625e+06])